# Full Example
THe idea with this NB is to do the same as the pymc_prophet_example NB but do everything (holidays, regressors etc...) in a single example and compare pymc_prohet with prophet. Then we will use this to build the Bayesian PyMC component.

In [1]:
import pandas as pd
from pymcprophet.model import BayesTS, BayesTSConfig
from prophet import Prophet

import numpy as np
import datetime as dt


Importing plotly failed. Interactive plots will not work.


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')
df['ds'] = pd.to_datetime(df['ds'])

## Holidays
These are user defined. We will also add auto US ones.

In [3]:
playoffs = pd.DataFrame({
  'holiday': 'playoff',
  'ds': pd.to_datetime(['2008-01-13', '2009-01-03', '2010-01-16',
                        '2010-01-24', '2010-02-07', '2011-01-08',
                        '2013-01-12', '2014-01-12', '2014-01-19',
                        '2014-02-02', '2015-01-11', '2016-01-17',
                        '2016-01-24', '2016-02-07']),
  'lower_window': 0,
  'upper_window': 1,
})
superbowls = pd.DataFrame({
  'holiday': 'superbowl',
  'ds': pd.to_datetime(['2010-02-07', '2014-02-02', '2016-02-07']),
  'lower_window': 0,
  'upper_window': 1,
})
holidays = pd.concat((playoffs, superbowls))

## Regressor
Custom dataset for Sundays during the NFL Season.

In [4]:
def nfl_sunday(ds):
    date = pd.to_datetime(ds)
    if date.weekday() == 6 and (date.month > 8 or date.month < 2):
        return 1
    else:
        return 0

df['nfl_sunday'] = df['ds'].apply(nfl_sunday)

## Modelling

In [5]:
train_df = df.query("ds.dt.year <= 2014")
test_df = df.query("ds.dt.year > 2014")

### PyMC

In [6]:
ts_config = BayesTSConfig()
ts_model = BayesTS(ts_config)
ts_model.add_holiday_country("US")
ts_model.add_holiday(holidays)
ts_model.add_regressor("nfl_sunday")

ts_model.assign_model_matrix(train_df)

In [15]:
ts_model.config.changepoints

[datetime.date(2008, 3, 2),
 datetime.date(2008, 5, 24),
 datetime.date(2008, 9, 4),
 datetime.date(2008, 11, 25),
 datetime.date(2009, 2, 14),
 datetime.date(2009, 5, 6),
 datetime.date(2009, 7, 25),
 datetime.date(2009, 10, 21),
 datetime.date(2010, 1, 12),
 datetime.date(2010, 4, 6),
 datetime.date(2010, 6, 27),
 datetime.date(2010, 9, 22),
 datetime.date(2010, 12, 11),
 datetime.date(2011, 3, 2),
 datetime.date(2011, 5, 22),
 datetime.date(2011, 8, 10),
 datetime.date(2011, 11, 1),
 datetime.date(2012, 1, 23),
 datetime.date(2012, 4, 12),
 datetime.date(2012, 7, 3),
 datetime.date(2012, 9, 22),
 datetime.date(2012, 12, 11),
 datetime.date(2013, 3, 2),
 datetime.date(2013, 5, 22),
 datetime.date(2013, 8, 12)]

In [16]:
ts_model.t_values

[0.0321955003878976,
 0.0643910007757952,
 0.104344453064391,
 0.13615205585725368,
 0.16757176105508145,
 0.19899146625290923,
 0.2300232738557021,
 0.26415826221877425,
 0.29635376260667184,
 0.32893716058960437,
 0.360744763382467,
 0.39449185415050425,
 0.42552366175329714,
 0.4569433669511249,
 0.4883630721489527,
 0.5193948797517456,
 0.5515903801396431,
 0.5837858805275408,
 0.6148176881303336,
 0.6466252909231963,
 0.6780449961210241,
 0.7090768037238169,
 0.7404965089216446,
 0.7719162141194724,
 0.8037238169123352]

### Prophet

In [9]:
prophet_model = Prophet(holidays=holidays)
prophet_model.add_country_holidays(country_name='US')
prophet_model.add_regressor('nfl_sunday')

In [10]:
transformed = prophet_model.preprocess(train_df)

In [11]:
transformed.t[75:85]

75    0.029480
76    0.029868
77    0.030256
78    0.030644
79    0.031420
80    0.032196
81    0.033359
82    0.033747
83    0.034135
84    0.034523
Name: t, dtype: float64

In [12]:
transformed.t_change

array([0.03335919, 0.064391  , 0.10434445, 0.13653995, 0.16757176,
       0.19899147, 0.23002327, 0.26415826, 0.29635376, 0.32893716,
       0.36074476, 0.39449185, 0.42552366, 0.45694337, 0.48836307,
       0.51939488, 0.55159038, 0.58378588, 0.61481769, 0.64662529,
       0.6776571 , 0.7090768 , 0.74049651, 0.77152832, 0.80333592])

In [13]:
np.linspace(0, 0.8, 26)[1:]

array([0.032, 0.064, 0.096, 0.128, 0.16 , 0.192, 0.224, 0.256, 0.288,
       0.32 , 0.352, 0.384, 0.416, 0.448, 0.48 , 0.512, 0.544, 0.576,
       0.608, 0.64 , 0.672, 0.704, 0.736, 0.768, 0.8  ])

In [14]:
transformed.tau

0.05